In [1]:
if Sys.iswindows()
    username = "Alex"
    pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
    using Pkg
    Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
    using StaticArrays, DifferentialEquations, DynamicalSystems, BenchmarkTools
        include("C:\\Users\\Alex\\Desktop\\dynamical-systems\\system.jl");
    #include("C:\\Users\\Alex\\Desktop\\dynamical-systems\\visual.jl");
else
    username = "nova"
    pathtorepo = "/home/nova/work/repo_ds/dynamical-systems"
    using Pkg
    Pkg.activate(pathtorepo * "/env/integrate/")
    using StaticArrays, DifferentialEquations, DynamicalSystems
    include("/home/nova/work/repo_ds/dynamical-systems/system.jl")
    #include("/home/nova/work/repo_ds/dynamical-systems/visual.jl")
end

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`


TM6_glial_ECM_help (generic function with 1 method)

In [2]:
tLSE = 3000.0; tt = 0.0; tstep = 0.001;
tol = 1e-15;
integ_set = (alg = Vern9(), adaptive = true, abstol = tol, reltol = tol);

In [3]:
param = TM6_glial_ECM_get_params();
dict, indexparams = TM6_glial_ECM_help(param);
println(indexparams);


    τ - 1, τD - 2, τF - 3, τy - 4, α - 5, αE - 6, αecm - 7, αp - 8,
    J - 9, U0 - 10, I0 - 11, ΔU0 - 12, β - 13, βecm  -14, βp - 15,
    γp - 16, ecm0 - 17, ecm1 - 18, kecm - 19, θecm - 20, p0 - 21, p1 - 22, θp - 23, kp - 24, ythr - 25, xthr - 26


In [4]:
u0 = [9.64476220646216, 0.4573265927731924, 0.8202126784574711, 0.0001125391840336231, 0.028243108855657985, 0.0006195369939899594]
param[11] =  -1.741;
param[6] = 2.757727272727273;

In [11]:
function find_zero(Λs; tol = 1e-2)
    vec_bool = Vector{Bool}(undef, 6);
    for (index, lp) in enumerate(Λs)
        checknull=  isapprox(lp, 0.0; atol = tol);
        vec_bool[index] = checknull;
    end
    vec_of_true = findall(x->x==true, vec_bool);
    if length(vec_of_true) >=1
        return 0;
    else
        return -1;
    end;
end

function lyapunovspectrum_adapt(ds, time_lse; tol_ = 1e-2)
    coeff_adapt_time = 1;
    flag = true;
    tands = TangentDynamicalSystem(ds; J = TM6_glial_ECM_jac);
    Λs = lyapunovspectrum(tands, time_lse);
    while flag == true
        checkzero = find_zero(Λs; tol = tol_);
        if checkzero == -1
            coeff_adapt_time*=2;
            Λs = lyapunovspectrum(tands, time_lse * coeff_adapt_time*10);
        else
            flag = false;
        end
    end
    return Λs, coeff_adapt_time;
end

lyapunovspectrum_adapt (generic function with 1 method)

In [48]:
function print_index_param(ds, name_param, index_param)
    println("u0: $(current_state(ds))");
    println("$(name_param): $(current_parameters(ds)[index_param])")
end

print_index_param (generic function with 1 method)

In [61]:
ds= CoupledODEs(TM6_glial_ECM, u0, param, diffeq = integ_set);
tands = TangentDynamicalSystem(ds; J = TM6_glial_ECM_jac);

systems = [deepcopy(tands) for _ in 1:Threads.nthreads()-1];
pushfirst!(systems, tands);

In [74]:
set_state!(ds, [1.0, 0.4, 0.7, 0.4, 0.4, 0.4]);
set_parameter!(ds, 6, 0.2);

In [125]:
Threads.nthreads()

6

In [13]:
@inbounds function TM6_glial_ECM_jac_(vars, params, t)

    E_sum =  params[9] + params[6] * vars[5];
    E_mult_x_mult_u = vars[3] * vars[2] * vars[1]

    exp_ = exp( ( E_sum * E_mult_x_mult_u + params[11] ) / params[5] );
    
    denomE = 1.0 + exp_;
    y_minus_ythr = vars[4] - params[25];
    E_sum_mult_exp_ = E_sum * exp_;
    E_sum_mult_exp_mult_E = E_sum_mult_exp_ * vars[1];
    

    exp50 = exp( -50.0 * ( y_minus_ythr ) );
    exp20 = exp( -20.0 * ( vars[2] - params[26] ) );
    exphecm = exp( -(vars[1] - params[20]) / (params[19]) );
    exphp = exp( -(vars[1] - params[23]) / (params[24]) );

    U = params[10] + params[12] / ( 1.0 + exp( -50.0 * ( y_minus_ythr ) ) );
    Uder = (50.0 * params[12] * exp50) / ( 1.0 + exp50 )^2;

    Hyder = 20.0 * exp20 / (1.0 + exp20)^2;

    Hecmder = (params[17] - params[18]) * exphecm / ( params[19] * (1.0 + exphecm)^2 );
    Hpder = (params[21] - params[22]) * exphp / ( params[24] * (1.0 + exphp)^2 );


    EE = ( -1.0 + ( E_sum_mult_exp_ * vars[3] * vars[2] ) / denomE ) / params[1];
    Ex = ( ( E_sum_mult_exp_mult_E * vars[3]  ) / denomE ) / params[1];
    Eu = ( ( E_sum_mult_exp_mult_E * vars[2]  ) / denomE ) / params[1];
    Eecm = ( (exp_ * params[6] * E_mult_x_mult_u) / denomE ) / params[1];


    xE = -vars[3] * vars[2];
    xx = - 1.0 / params[2] - vars[3] * vars[1];
    xu = -vars[2] * vars[1];


    uE = U * (1.0 - vars[3]);
    uu = -1.0 / params[3] - U * vars[1];
    uy = Uder * ( 1.0 / params[3] + (1.0 - vars[3]) * vars[1] );

    yx = params[13] * Hyder;

    ecmE =  -params[14] * Hecmder;
    ecmecm = -(params[7] + params[16] * vars[6]);
    ecmp = -vars[5] * params[16];

    pE = -params[15] * Hpder;

    SMatrix{6,6}(EE, xE, uE, 0.0, ecmE, pE,
        Ex, xx, 0.0, yx, 0.0, 0.0,
        Eu, xu, uu, 0.0, 0.0, 0.0,
        0.0, 0.0, uy, - 1.0 / params[4], 0.0, 0.0,
        Eecm, 0.0, 0.0, 0.0, ecmecm, 0.0,
        0.0, 0.0, 0.0, 0.0, ecmp, -params[8])
end

TM6_glial_ECM_jac_ (generic function with 1 method)